In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../20230505_Messages.csv")
df.head()

In [ ]:
df["Message"][21]

In [ ]:
# Group by week
df["Datetime"] = pd.to_datetime(df["Datetime"])
df['Week'] = df['Datetime'].dt.isocalendar().week
df['Date'] = df['Datetime'].dt.date

# Group by Date
daily_df = df.groupby('Date').agg({'Message': ' \n '.join}).reset_index()
daily_df = pd.DataFrame(daily_df)
len(daily_df)

# # Group by Week
# weekly_df = df.groupby('Week').agg({'Message': ' \n '.join}).reset_index()
# weekly_df = pd.DataFrame(messages_df)
# print(weekly)
# print(weekly_df["Message"][9])

In [ ]:
daily_df["wc"] = daily_df["Message"].apply(lambda x: len(x.split()))

In [ ]:
daily_df["wc"].describe()

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

from functools import lru_cache
llm = ChatOpenAI(temperature=0)

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder()

In [ ]:
%%time
class SummarizeDay:
    def __init__(self, plain_text):
        self.prompt_template = """This is a chaotic Generative AI Group Chat transcript. Write detailed, exhaustive bullet point recap of topics discussed. Extract COMPLETE URL of web and social links with context. Please organise it into sections, only when needed:

{text}


Use Markdown. Add ## for section titles. TOPICS RECAP:"""
# Research with weblinks where relevant EXACTLY ONCE:
        self.PROMPT = PromptTemplate(template=self.prompt_template, input_variables=["text"])
#         self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
        self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=self.PROMPT)
        self.docs = self.make_docs(plain_text)
        
    @lru_cache
    def make_docs(self, plain_text: str):
        texts = text_splitter.split_text(plain_text)
        docs = [Document(page_content=t) for t in texts]
        return docs

    def summarize_docs(self):   
        chain_output = self.chain({"input_documents": self.docs}, return_only_outputs=True)
        return chain_output

plain_text = daily_df["Message"][36]
sd = SummarizeDay(plain_text)
chain_output = sd.summarize_docs()
output_text = chain_output["output_text"]
print(output_text)

In [ ]:
import re
WINDOW = 1
def extract_urls_with_context(text):
    lines = text.split('\n')
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    urls_with_context = []

    for idx, line in enumerate(lines):
        for match in url_pattern.finditer(line):
            start, end = match.span()
            prev_line = lines[idx - WINDOW] if idx > 0 else ""
            next_line = lines[idx + WINDOW] if idx < len(lines) - 1 else ""
            context = f"{prev_line}\n{line}\n{next_line}".strip()
            urls_with_context.append((match.group(), context))

    return urls_with_context


    urls_with_context = extract_urls_with_context(url)
    for url, context in urls_with_context:
        print(f"URL: {url}")
        print(f"Context: {context}\n")
        


class LinksContext:
    def __init__(self, plain_text):
        self.prompt_template = """For each URL, there is some context. Newlines may or may not be related to the link, but the message in the same link as link is related to the link.
        
{text}
        
Mention URL with context. Single bullet point:"""
# Research with weblinks where relevant EXACTLY ONCE:
        self.PROMPT = PromptTemplate(template=self.prompt_template, input_variables=["text"])
#         self.chain = load_summarize_chain(ChatOpenAI(temperature=0, model_name="gpt-4"), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
        self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=self.PROMPT)
        self.docs = self.make_docs(plain_text)
        
    @lru_cache
    def make_docs(self, plain_text: str):
        texts = text_splitter.split_text(plain_text)
        docs = [Document(page_content=t) for t in texts]
        return docs

    def summarize_docs(self):   
        chain_output = self.chain({"input_documents": self.docs}, return_only_outputs=True)
        return chain_output

plain_text = daily_df["Message"][54]
url_groups = extract_urls_with_context(plain_text)
for ug in url_groups:
    lc = LinksContext(ug[1])
    chain_output = lc.summarize_docs()
    output_text = chain_output["output_text"]
    print(output_text)